## ***DATA CLEANING AND PREPROCESSING***

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import scipy as sp
import scipy.stats

In [2]:
df = pd.read_excel("retail.xlsx", engine='openpyxl')
df.head(5)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   InvoiceNo    541909 non-null  object        
 1   StockCode    541909 non-null  object        
 2   Description  540455 non-null  object        
 3   Quantity     541909 non-null  int64         
 4   InvoiceDate  541909 non-null  datetime64[ns]
 5   UnitPrice    541909 non-null  float64       
 6   CustomerID   406829 non-null  float64       
 7   Country      541909 non-null  object        
dtypes: datetime64[ns](1), float64(2), int64(1), object(4)
memory usage: 33.1+ MB


In [3]:
print("Number of rows in dataset is: ", df.shape[0])
print("Number of columns in dataset is: ", df.shape[1])

Number of rows in dataset is:  541909
Number of columns in dataset is:  8


In [4]:
#Finding the unique values of each attribute
print("Number of transactions: ", df['InvoiceNo'].nunique())
print("Number of products bought: ",df['StockCode'].nunique())
print("Number of customers:", df['CustomerID'].nunique() )
print('Number of countries: ',df['Country'].nunique())

Number of transactions:  25900
Number of products bought:  4070
Number of customers: 4372
Number of countries:  38


In [5]:
df.describe()

,Quantity,UnitPrice,CustomerID
count,541909.000000,541909.000000,406829.000000
mean,9.552250,4.611114,15287.690570
std,218.081158,96.759853,1713.600303
min,-80995.000000,-11062.060000,12346.000000
25%,1.000000,1.250000,13953.000000
50%,3.000000,2.080000,15152.000000
75%,10.000000,4.130000,16791.000000
max,80995.000000,38970.000000,18287.000000


***Here, in the descriptive statistics, we see the presence of negative values for "Quantity" and "UnitPrice" which is not reasonable.***

In the dataset, it is mentioned that the Invoice of cancelled orders start with 'C'. Hence, we filter out the cancelled orders.

In [6]:
cancelled = df[df['InvoiceNo'].astype(str).str.contains('C')]
cancelled.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
141,C536379,D,Discount,-1,2010-12-01 09:41:00,27.50,14527.0,United Kingdom
154,C536383,35004C,SET OF 3 COLOURED FLYING DUCKS,-1,2010-12-01 09:49:00,4.65,15311.0,United Kingdom
235,C536391,22556,PLASTERS IN TIN CIRCUS PARADE,-12,2010-12-01 10:24:00,1.65,17548.0,United Kingdom
236,C536391,21984,PACK OF 12 PINK PAISLEY TISSUES,-24,2010-12-01 10:24:00,0.29,17548.0,United Kingdom
237,C536391,21983,PACK OF 12 BLUE PAISLEY TISSUES,-24,2010-12-01 10:24:00,0.29,17548.0,United Kingdom


In [7]:
cancelled[cancelled['Quantity']>0]

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country


In [8]:
print("Number of cancelled transactions:", len(cancelled))
print("Total number of transaction with negative quantity:", df[df['Quantity'] <= 0 ]['Quantity'].count())

Number of cancelled transactions: 9288
Total number of transaction with negative quantity: 10624


***There are no cancelled transactions with positive quantity. So we assume cancelled transactions to have negative quantity. However, in additional to the cancelled transactions, there are some extra negative quantity transactions that do not have 'C' in their InvoiceNo. This means that these products were not cancelled either. As this makes no sense, we are going to filter and keep only the cancelled transactions. It is to be mentioned that we could have deleted all the negative quantity transactions from our dataset as they did not generate any sales. But as part of our final objective, we are keeping the cancelled transactions in the dataset as an indicator of the customer loyalty.***

In [9]:
# filtered_cancelled = df[(df['Quantity'] <= 0) & ~(df['InvoiceNo'].astype(str).str.contains('C'))]['InvoiceNo'].index
# df = df.drop(filtered_cancelled).reset_index(drop=True)
# df.shape

df.drop(df[(df.Quantity>0) & (df.InvoiceNo.str.contains('C') == True)].index, inplace = True)
df.drop(df[(df.Quantity<0) & (df.InvoiceNo.str.contains('C') == False)].index, inplace = True)

***There are also transactions where the UnitPrice is zero. As there is no clarity in these information, we are also dropping them. In order to remove outliers in both the UnitPrice and Quantity features, we also perform z score analysis. Hence this helps us to handle the outliers and filter both the features in our dataset. Note that the cancelled transactions have negative quantity.***

In [10]:
# df = df[(np.abs(sp.stats.zscore(df['UnitPrice']))<3) & (np.abs(sp.stats.zscore(df['Quantity']))<5)]
zero = df[df.UnitPrice == 0].count()
print("Total observations with zero Unit Price:", zero.UnitPrice)

Total observations with zero Unit Price: 2515


In [11]:

df.drop(df[df.UnitPrice == 0].index, inplace = True)

In [12]:
df = df[(np.abs(sp.stats.zscore(df['UnitPrice']))<3) & (np.abs(sp.stats.zscore(df['Quantity']))<5)]


In [13]:
df.shape[0]

538922

In [14]:
# df2 = df[(df['UnitPrice'] > 0) & (df['Quantity']>0)]
df2 = df

In [15]:
df2.describe()

,Quantity,UnitPrice,CustomerID
count,538922.000000,538922.000000,406565.000000
mean,9.592114,3.537128,15288.052243
std,28.889910,7.510164,1713.481792
min,-960.000000,0.001000,12347.000000
25%,1.000000,1.250000,13954.000000
50%,3.000000,2.080000,15152.000000
75%,10.000000,4.130000,16791.000000
max,1056.000000,295.200000,18287.000000


***Now we can see that there are no negative values in UnitPrice and the dataset is refined from unfulfilled transactions. The negative value in Quantity is because we didn't drop the cancelled transactions as we want to identify such customer segments as well who cancel the orders and do not contribute to the revenue of the company.***

In [16]:
#To check if there are any missing values in the dataset
df2.isnull().sum()

InvoiceNo           0
StockCode           0
Description         0
Quantity            0
InvoiceDate         0
UnitPrice           0
CustomerID     132357
Country             0
dtype: int64

***There were missing values in the "Description" and "CustomerID" column. After removing the problematic transactions, all the missing "Description" observations were also removed.***

***As our objective is customer segmentation, so the missing "CustomerID" needs to be filled up if possible. We will try to fill up the missing CustomerID mapping with InvoiceNo. or else delete it.***

In [17]:
# Checking whether the InvoiceNo and CustomerID have one to many relationship or not. If not,
# we can fill the missing CustomerID with the help of unique InvoiceNo.

def checking_one_to_many(df, col1, col2): 
    first = df.drop_duplicates([col1, col2]).groupby(col1)[col2].count().max()
    print(first == 1)
    
checking_one_to_many(df2, 'InvoiceNo', 'CustomerID')
#checking_one_to_one(df, 'CustomerID', 'InvoiceNo')


True


***Here, the InvoiceNo has one to many relationship with the CustomerID. It is not possible to fill up the missing CustomerID using the InvoiceNo. Because multiple CustomerID may have the same InvoiceNo.  So, considering we have enough data, we will remove the observations with missing CustomerID.***

In [18]:
df2 = df2.drop(df2[df2.CustomerID.isnull()].index).reset_index(drop=True)
df2.isnull().sum()

InvoiceNo      0
StockCode      0
Description    0
Quantity       0
InvoiceDate    0
UnitPrice      0
CustomerID     0
Country        0
dtype: int64

In [19]:
df2.shape[0]

406565

***So we have removed all the missing values from the dataset***

In [20]:
df2['InvoiceNo'] = df2.InvoiceNo.astype('str')
df2['Description'] = df2.Description.astype('str')
df2['Description'] = df2.Description.str.strip()

checking_one_to_many(df2,'StockCode','Description')

False


**Generally, StockCode should have one to many relationship with Description. It means each StockCode needs to uniquely represent an item Description, but each Description may not have just one StockCode. However, we find the relation to be False. This is because the dataset has multiple Descriptions for the same StockCode.
So, we will keep only one Description for the same StockCode and make the one to many relation True.**

In [21]:
products = df2.groupby(['StockCode','Description'],as_index=False).count()
products = products.loc[:,['StockCode','Description']]
stockCount = products.StockCode.value_counts().to_frame()
stockList = list(stockCount[stockCount.StockCode>1].index) #List of stockCode with more than 1 Description
products[products.StockCode.isin(stockList)].head(6)

,StockCode,Description
48,20622,VIP PASSPORT COVER
49,20622,VIPPASSPORT COVER
101,20725,LUNCH BAG RED RETROSPOT
102,20725,LUNCH BAG RED SPOTTY
194,20914,SET/5 RED RETROSPOT LID GLASS BOWLS
195,20914,SET/5 RED SPOTTY LID GLASS BOWLS


In [22]:
products.drop_duplicates('StockCode', keep='first', inplace=True)

In [23]:

# Correct the mapping of StockCode and Description
df2 = pd.merge(df2,products,on='StockCode', how='left',validate='many_to_one')
df2.drop('Description_x',axis=1,inplace=True)
df2.rename(columns={'Description_y':'Description'},inplace=True)
df2.groupby(['StockCode','Description'],as_index=False).head(100)

,InvoiceNo,StockCode,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,Description
0,536365,85123A,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom,CREAM HANGING HEART T-LIGHT HOLDER
1,536365,71053,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,WHITE METAL LANTERN
2,536365,84406B,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom,CREAM CUPID HEARTS COAT HANGER
3,536365,84029G,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,KNITTED UNION FLAG HOT WATER BOTTLE
4,536365,84029E,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,RED WOOLLY HOTTIE WHITE HEART.
...,...,...,...,...,...,...,...,...
406520,581582,23498,12,2011-12-09 12:21:00,1.45,17581.0,United Kingdom,CLASSIC BICYCLE CLIPS
406523,581584,20832,72,2011-12-09 12:25:00,0.72,13777.0,United Kingdom,RED FLOCK LOVE HEART PHOTO FRAME
406525,581585,22481,12,2011-12-09 12:31:00,0.39,15804.0,United Kingdom,BLACK TEA TOWEL CLASSIC DESIGN
406537,581585,16016,10,2011-12-09 12:31:00,0.85,15804.0,United Kingdom,LARGE CHINESE STYLE SCISSOR


In [24]:
# Check for StockCode and Description mapping
checking_one_to_many(df2,'StockCode','Description')

True


***In the dataset description, it is also mentioned that the StockCode is the product code containing 5 digit integer. However, there are StockCode containing more or less number of codes than 5. We need to analyse that and determine whether to keep them or not.***

In [25]:
df2[(df2.StockCode.str.len())==6].sort_values(by='StockCode').head(10)

,InvoiceNo,StockCode,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,Description
72630,545655,10123C,3,2011-03-04 13:58:00,0.65,14670.0,United Kingdom,HEARTS WRAPPING TAPE
96361,548491,10123C,1,2011-03-31 13:14:00,0.65,14064.0,United Kingdom,HEARTS WRAPPING TAPE
4161,536863,10123C,1,2010-12-03 11:19:00,0.65,17967.0,United Kingdom,HEARTS WRAPPING TAPE
333003,574686,10124A,4,2011-11-06 13:00:00,0.42,17608.0,United Kingdom,SPOTS ON RED BOOKCOVER TAPE
53945,543397,10124A,3,2011-02-08 10:32:00,0.42,17859.0,United Kingdom,SPOTS ON RED BOOKCOVER TAPE
8981,537382,10124A,4,2010-12-06 13:13:00,0.42,16710.0,United Kingdom,SPOTS ON RED BOOKCOVER TAPE
92085,547881,10124A,1,2011-03-27 16:08:00,0.42,13110.0,United Kingdom,SPOTS ON RED BOOKCOVER TAPE
83238,546913,10124A,4,2011-03-17 20:18:00,0.42,15861.0,United Kingdom,SPOTS ON RED BOOKCOVER TAPE
83237,546913,10124G,4,2011-03-17 20:18:00,0.42,15861.0,United Kingdom,ARMY CAMO BOOKCOVER TAPE
333002,574686,10124G,4,2011-11-06 13:00:00,0.42,17608.0,United Kingdom,ARMY CAMO BOOKCOVER TAPE


In [26]:
df2[(df2.StockCode.str.len())<5].sort_values(by='StockCode').head()

,InvoiceNo,StockCode,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,Description
163942,556969,C2,1,2011-06-16 10:11:00,25.0,16257.0,United Kingdom,CARRIAGE
145853,554770,C2,1,2011-05-26 11:40:00,50.0,14911.0,EIRE,CARRIAGE
146235,554826,C2,2,2011-05-26 14:53:00,18.0,12493.0,France,CARRIAGE
25982,539688,C2,1,2010-12-21 11:00:00,150.0,12678.0,France,CARRIAGE
256283,567631,C2,1,2011-09-21 13:33:00,50.0,13350.0,United Kingdom,CARRIAGE


***The StockCode with 6 digit codes seem to distinguish the variants of a 5 digit integer with the last alphabet. There are also StockCodes with 2,3 and 4 digits that appear to be related to different charges related to the product. As these do not provide any helpful information about the product that the customer buys, we are discarding them from the dataset.***

In [27]:
df2 = df2.drop(df2[(df2.StockCode.str.len())<5].index).reset_index(drop=True)
df2.shape

(404780, 8)

***Mapping a specific StockCode with a specific UnitPrice based on the mode value for our convenience. It is because some StockCodes have different UnitPrice at different times based on discounts.*** 

In [28]:
#Incorrect Prices
StockList = df2.StockCode.unique()
CalculatedMode = map(lambda x: df2.UnitPrice[df2.StockCode == x].mode()[0],StockList)
StockModes = list(CalculatedMode)
for i,v in enumerate(StockList):
    df2.loc[df2['StockCode']== v, 'UnitPrice'] = StockModes[i]

In [29]:
# list_timestamp = list(df2.InvoiceDate)
# DateTime = pd.DataFrame(pd.to_datetime(list_timestamp), columns=['DateTime'])
# df2['DateTime'] = DateTime['DateTime'].values

In [30]:
df2.head()

,InvoiceNo,StockCode,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,Description
0,536365,85123A,6,2010-12-01 08:26:00,2.95,17850.0,United Kingdom,CREAM HANGING HEART T-LIGHT HOLDER
1,536365,71053,6,2010-12-01 08:26:00,3.75,17850.0,United Kingdom,WHITE METAL LANTERN
2,536365,84406B,8,2010-12-01 08:26:00,4.15,17850.0,United Kingdom,CREAM CUPID HEARTS COAT HANGER
3,536365,84029G,6,2010-12-01 08:26:00,4.25,17850.0,United Kingdom,KNITTED UNION FLAG HOT WATER BOTTLE
4,536365,84029E,6,2010-12-01 08:26:00,4.25,17850.0,United Kingdom,RED WOOLLY HOTTIE WHITE HEART.


***Some of the CustomerID belongs to people in two different Country. The probable reason may be due to data entry error or the person has moved from one country to another. Here, if any CustomerID has two Country, we replace the incorrect value with the mode value of the countries of the respective customer.***

In [31]:
customers = df2.groupby('CustomerID')['Country'].unique()
customers.loc[customers.apply(lambda x:len(x)>1)]

CustomerID
12370.0           [Cyprus, Austria]
12394.0          [Belgium, Denmark]
12417.0            [Belgium, Spain]
12422.0    [Australia, Switzerland]
12429.0          [Denmark, Austria]
12431.0        [Australia, Belgium]
12455.0             [Cyprus, Spain]
12457.0       [Switzerland, Cyprus]
Name: Country, dtype: object

In [32]:
#Fixing Duplicate CustomerIDs
for i,v in df2.groupby('CustomerID')['Country'].unique().items():
    if len(v)>1:
        df2.Country[df2['CustomerID'] == i] = df2.Country[df2['CustomerID'] == i].mode()[0]

D:\TOOLS\Anaconda\envs\tf\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [33]:
customers = df2.groupby('CustomerID')['Country'].unique()
customers.loc[customers.apply(lambda x:len(x)>1)]

Series([], Name: Country, dtype: object)

In [34]:
df2.shape

(404780, 8)

In [35]:
df = df2

***Now there are no customers belonging to more than one country for our convenience.***

# *FEATURE ENGINEERING: PART A*

***Adding a column for total value for each product. Note that, for cancelled orders, the negative quantity multiplied with the negative unit price gives positive total price for the transactions that were not completed.***

In [36]:
# Adding a column for total value for each product 
df['TotalPrice'] = df.Quantity*df.UnitPrice
df['TotalPrice'] = df['TotalPrice'].apply(lambda x: abs(x))
df.head()

,InvoiceNo,StockCode,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,Description,TotalPrice
0,536365,85123A,6,2010-12-01 08:26:00,2.95,17850.0,United Kingdom,CREAM HANGING HEART T-LIGHT HOLDER,17.7
1,536365,71053,6,2010-12-01 08:26:00,3.75,17850.0,United Kingdom,WHITE METAL LANTERN,22.5
2,536365,84406B,8,2010-12-01 08:26:00,4.15,17850.0,United Kingdom,CREAM CUPID HEARTS COAT HANGER,33.2
3,536365,84029G,6,2010-12-01 08:26:00,4.25,17850.0,United Kingdom,KNITTED UNION FLAG HOT WATER BOTTLE,25.5
4,536365,84029E,6,2010-12-01 08:26:00,4.25,17850.0,United Kingdom,RED WOOLLY HOTTIE WHITE HEART.,25.5


***We are adding a new column with boolean value 1 if any order was cancelled or else 0***

In [37]:
df['CancelledOrder'] = df.InvoiceNo.astype('str').str.startswith('C').astype('int')

***As this is a UK based store, we divide the countries into two classes. For others from above, we assign boolian value 1 or else 0.***

In [38]:
df['Place'] = df.Country.map(lambda x: 'Home' if x=='United Kingdom' else 'Abroad')
df['Place'].nunique()

2

In [39]:
df.head()

,InvoiceNo,StockCode,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,Description,TotalPrice,CancelledOrder,Place
0,536365,85123A,6,2010-12-01 08:26:00,2.95,17850.0,United Kingdom,CREAM HANGING HEART T-LIGHT HOLDER,17.7,0,Home
1,536365,71053,6,2010-12-01 08:26:00,3.75,17850.0,United Kingdom,WHITE METAL LANTERN,22.5,0,Home
2,536365,84406B,8,2010-12-01 08:26:00,4.15,17850.0,United Kingdom,CREAM CUPID HEARTS COAT HANGER,33.2,0,Home
3,536365,84029G,6,2010-12-01 08:26:00,4.25,17850.0,United Kingdom,KNITTED UNION FLAG HOT WATER BOTTLE,25.5,0,Home
4,536365,84029E,6,2010-12-01 08:26:00,4.25,17850.0,United Kingdom,RED WOOLLY HOTTIE WHITE HEART.,25.5,0,Home


In [40]:
df.to_csv('cleaned_dataset.csv', index = False)

In [41]:
df.shape

(404780, 11)